<a href="https://colab.research.google.com/github/LeeJungtae/TensorflowProject1/blob/main/TensorflowProject3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이번에는 텍스트 분류 실습을 해보려고 한다.

인터넷 영화 데이터베이스(IMDB)에서 수집한 50,000개의 영화 리뷰 텍스트를 담은 IMDB 데이터셋을 활용한다.\
25,000개의 리뷰는 훈련용, 25,000개는 테스트용으로 나뉘어져 있다.\
훈련 세트와 테스트 세트의 개수가 동일하므로 긍정과 부정의 개수가 동일하다.

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.4.1


IMDB 데이터셋은 텐서플로우와 함께 제공된다.\
리뷰(단어의 시퀀스(Sequence))는 미리 전처리해서 정수 시퀀스로 변환되어 있다.\
각 정수는 어휘 사전에 있는 특정 단어를 의미한다.

다음 코드는 IMDB 데이터셋을 컴퓨터에 다운로드한다.

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

매개변수 num_words=10000은 훈련 데이터에서 가장 많이 등장하는 상위 10,000개의 단어를 선택한다.\
데이터 크기를 적당하게 유지하기 위해 드물게 등장하는 단어는 제외했다.

이 데이터셋의 샘플은 전처리된 정수 배열이다.\
이 정수는 영화 리뷰에 나오는 단어를 나타낸다.\
레이블(label)은 정수 0 또는 1이다.\
0은 부정적인 리뷰, 1은 긍정적인 리뷰다.

In [3]:
print('훈련 샘플 : {}, 레이블 : {}'.format(len(train_data), len(train_labels)))

훈련 샘플 : 25000, 레이블 : 25000


리뷰 텍스트는 어휘 사전의 특정 단어를 나타내는 정수로 변환되어 있다.\
첫 번째 리뷰를 확인해보자.

In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


영화 리뷰들은 길이가 다르다.\
다음 코드는 첫 번째 리뷰와 두 번째 리뷰에서 단어의 개수를 출력한다.\
신경망의 입력은 길이가 같아야 하므로 이걸 해결해보도록 하자.

In [5]:
len(train_data[0]), len(train_data[1])

(218, 189)

정수를 다시 텍스트로 변환해보자.\
여기서는 정수와 문자열을 매핑한 딕셔너리(dictionary) 객체에 질의하는 헬퍼(Helper) 함수를 만들어보자.

In [6]:
# 단어와 정수 인덱스를 매핑한 딕셔너리
word_index = imdb.get_word_index()

# 처음 몇 개 인덱스는 사전에 정의되어 있다.
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2 # unknown
word_index['<UNUSED>'] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

1646592/1641221 [==============================] - 0s 0us/step


이제 decode_review 함수를 사용해 첫 번째 리뷰 텍스트를 출력해보자.

In [7]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

리뷰-정수 배열-는 신경망에 주입하기 전에 먼저 텐서로 변환되어야 한다.\
변환 방법에는 몇 가지가 있다.

- 원-핫 인코딩(one-hot encoding)은 정수 배열을 0과 1로 이뤄진 벡터로 변환한다. 예를 들어 배열 [3, 5]를 인덱스 3과 5만 1이고 나머지는 0인 10,000차원 벡터로 변환할 수 있다. 그 다음 실수 벡터 데이터를 다룰 수 있는 층-Dense 층-을 신경망의 첫 번째 층으로 사용한다. 이 방법은 num_words * num_reviews 크기의 행렬이 필요하기 때문에 메모리를 많이 사용한다.

- 다른 방법으로는, 정수 배열의 길이가 모두 같도록 패딩(padding)을 추가해 max_length * num_reviews 크기의 정수 텐서를 만든다. 이런 형태의 텐서를 다룰 수 있는 임베딩(embeddding) 층을 신경망의 첫 번째 층으로 사용할 수 있다.

여기서는 두 번째 방식을 사용해보겠다.\
영화 리뷰의 길이가 같아야 하므로 pad_sequence 함수를 사용해 길이를 맞추자.

In [10]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index['<PAD>'],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=256)

샘플의 길이를 확인해보자.

In [11]:
len(train_data[0]), len(train_data[1])

(256, 256)

(패딩된) 첫 번째 리뷰 내용을 확인해보자.

In [12]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

신경망은 층(layer)을 쌓아서 만든다.\
이 구조에서는 두 가지를 결정해야 한다.

- 모델에서 얼마나 많은 층을 사용할 것인가?

- 각 층에서 얼마나 많은 은닉 유닛(hidden unit)을 사용할 것인가?

이 예제의 입력 데이터는 단어 인덱스의 배열이다.\
예측할 레이블은 0 또는 1이다.\
이 문제에 맞는 모델을 구성해보자.

In [13]:
# 입력 크기는 영화 리뷰 데이터셋에 적용된 어휘 사전의 크기이다.(10,000개의 단어)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


층을 순서대로 쌓아 분류기(classifier)를 만든다.

1. 첫 번째 층은 Embedding 층이다. 이 층은 정수로 인코딩된 단어를 입력받고 각 단어 인덱스에 해당하는 임베딩 벡터를 찾는다. 이 벡터는 모델이 훈련되면서 학습된다. 이 벡터는 출력 배열에 새로운 차원으로 추가된다. 최종 차원은 (batch, sequence, embedding)이 된다.

2. 그 다음 GlobalAveragePooling1D 층은 sequence 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환한다. 이는 길이가 다른 입력을 다루는 가장 간단한 방법이다.

3. 이 고정 길이의 출력 벡터는 16개의 은닉 유닉을 가진 완전 연결(fully-connected) 층(Dense)을 거친다.

4. 마지막 층은 하나의 출력 노드(node)를 가진 완전 연결 층이다. sigmoid 활성화 함수를 사용하여 0과 1 사이의 실수를 출력한다. 이 값은 확률 또는 신뢰도를 나타낸다.

위 모델에는 입력과 출력 사이에 두 개의 중간 또는 "은닉" 층이 있다.\
출력(유닛 또는 노드, 뉴런)의 개수는 층이 가진 표현 공간(representational space)의 차원이 된다.\
이는 내부 표현을 학습할 때 허용되는 네트워크 자유도의 양과 동일하다.

모델에 많은 은닉 유닛(고차원의 표현 공간)과 층이 있다면 네트워크는 더 복잡한 표현을 학습할 수 있다.\
하지만 네트워크의 계산 비용이 많이 들고 원치않는 패턴을 학습할 수도 있다.\
이런 표현은 훈련 데이터의 성능을 향상시키지만 테스트 데이터에서는 그렇지 못 한다.\
이를 과대적합(overfitting)이라고 부른다.

모델이 훈련하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요하다.\
이 예제는 이진 분류 문제이고(긍정 또는 부정) 모델이 확률을 출력하므로 binary_crossentropy 손실 함수를 사용하겠다.

이제 모델이 사용할 옵티마이저와 손실 함수를 설정해보자.

In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

모델을 훈련할 때 모델이 만난 적 없는 데이터에서 정확도를 확인하는 것이 좋다.\
원본 훈련 데이터에서 10,000개의 셈플을 떼어내어 검증 세트(validation set)를 만들자.

In [15]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

이 모델을 512개의 샘플로 이뤄진 미니배치(mini-batch)에서 40번의 에포크(epoch) 동안 훈련한다.\
x_train과 y_train 텐서에 있는 모든 샘플에 대해 40번 반복한다는 뜻이다.\
훈련하는 동안 10,000개의 검증 세트에서 모델의 손실과 정확도를 모니터링한다.

In [16]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 2s 27ms/step - loss: 0.6927 - accuracy: 0.5372 - val_loss: 0.6898 - val_accuracy: 0.6750
Epoch 2/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6878 - accuracy: 0.7232 - val_loss: 0.6815 - val_accuracy: 0.7448
Epoch 3/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6765 - accuracy: 0.7647 - val_loss: 0.6650 - val_accuracy: 0.7501
Epoch 4/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6559 - accuracy: 0.7801 - val_loss: 0.6386 - val_accuracy: 0.7580
Epoch 5/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6250 - accuracy: 0.7766 - val_loss: 0.6020 - val_accuracy: 0.7794
Epoch 6/40
30/30 [==============================] - 1s 19ms/step - loss: 0.5828 - accuracy: 0.8097 - val_loss: 0.5586 - val_accuracy: 0.8035
Epoch 7/40
30/30 [==============================] - 1s 20ms/step - loss: 0.5319 - accuracy: 0.8281 - val_loss: 0.5131 - val_accuracy: 0.8184
Epoch 8/40
30

모델의 성능을 확인해보자.\
두 개의 값이 반환된다.\
손실(오차를 나타내는 숫자이므로 낮을 수록 좋다)과 정확도이다.

In [17]:
results = model.evaluate(test_data, test_labels, verbose=2)
print(results)

782/782 - 1s - loss: 0.3345 - accuracy: 0.8720
[0.3345036208629608, 0.8719599843025208]


본 예제는 매우 단순한 방식을 사용하므로 87%의 정확도를 달성했다.\
고급 방법을 사용한 모델은 95%에 가까운 정확도를 얻는다.

model.fit()은 History 객체를 반환한다.\
여기에는 훈련하는 동안 일어난 모든 정보가 담긴 딕셔너리(dictionary)가 들어있다.

In [18]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

네 개의 항목이 있다.\
훈련과 검증 단계에서 모니터링하는 지표들이다.\
훈련 손실과 검증 손실을 그래프로 그려보고, 훈련 정확도와 검증 정확도도 그래프로 그려서 비교해보자.

In [19]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo"는 '파란색 점'이다.
plt.plot(epochs, loss, 'bo', label='Training loss')
# b는 '파란 실선'이다.
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [20]:
plt.clf()   # 그림을 초기화한다.

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

이 그래프에서 점선은 훈련 손실과 훈련 정확도를 나타낸다.\
실선은 검증 손실과 검증 정확도이다.

훈련 손실은 에포크마다 감소하고 훈련 정확도는 증가한다는 것을 주목하자.\
경사 하강법 최적화를 사용할 때 볼 수 있는 현상이다.\
매 반복마다 최적화 대상의 값을 최소화한다.

하지만 검증 손실과 검증 정확도에서는 그러지 못 한다.\
약 20번째 에포크 이후가 최적점인 것 같다.\
이는 과대적합 때문이다.\
이전에 본 적 없는 데이터보다 훈련 데이터에서 더 잘 동작한다.\
이 지점부터는 모델이 과도하게 최적화되어 테스트 데이터에서 일반화되기 어려운 훈련 데이터의 특정 표현을 학습한다.

여기서는 과대적합을 막기 위해 단순히 20번째 에포크 근처에서 훈련을 멈출 수 있다.